# Data Assembly

In [1]:
import pandas as pd
import numpy as np

In [2]:
DJI = pd.read_csv(
    '~/Git/MachineLearningAndBigDataWithR/Data/^DJIMonthly.csv'
    , sep = ',')
SP500 = pd.read_csv(
    '~/Git/MachineLearningAndBigDataWithR/Data/^GSPCMonthly.csv'
    , sep = ',')
FedFunds = pd.read_csv(
    '~/Git/MachineLearningAndBigDataWithR/Data/FEDFUNDS.csv'
    , sep = ',')
SS = pd.read_csv(
    '~/Git/MachineLearningAndBigDataWithR/Data/SSRetired.csv'
    , sep = ',')
CPI = pd.read_csv(
    '~/Git/MachineLearningAndBigDataWithR/Data/CPIAUCSL.csv'
    , sep = ',')

### DJI
https://finance.yahoo.com/quote/%5EDJI/history?period1=475826400&period2=1542520800&interval=1mo&filter=history&frequency=1mo

In [3]:
#DJI.head()
DJI.columns = ['date', 'DJIopen', 'DJIhigh', 'DJIlow', 'DJIclose', 'DJIadjClose', 'DJIvolume']
#DJI.head()

### S&P 500
https://finance.yahoo.com/quote/%5EGSPC/history?period1=-630957600&period2=1542520800&interval=1mo&filter=history&frequency=1mo

In [4]:
#SP500.head()
SP500.columns = ['date', 'SPopen', 'SPhigh', 'SPlow', 'SPclose', 'SPadjClose', 'SPvolume']
#SP500.head()

df = pd.merge(DJI,SP500, how = 'inner', on = 'date')
#df.head()

### Federal funds rate
https://fred.stlouisfed.org/series/FEDFUNDS

In [5]:
#FedFunds.head()
FedFunds.columns = ['date', 'fedFundRate']
#FedFunds.head()
df = pd.merge(df,FedFunds, how = 'inner', on = 'date')
#df.head()

### Retired social security filings
https://www.ssa.gov/OACT/ProgData/awards.html

In [6]:
#SS.head()
SS.columns = ['date', 'totalSSRetired', 'averageSSRetiredPay', 'totalMaleSSRetired', 'averageMaleSSRetiredPay', 'totalFemaleSSRetired', 'averageFemaleSSRetiredPay', 'DROPME']
SS.drop('DROPME', axis = 1, inplace=True)
#SS.head()

adjDate = SS['date'].str.split("-", n = 1, expand = True)
#adjDate.head()
adjDate['month'] = np.where(adjDate[0] == 'Jan', '-01-01'
                               , np.where(adjDate[0] == 'Feb', '-02-01'
                                          , np.where(adjDate[0] == 'Mar', '-03-01'
                                                    ,np.where(adjDate[0] == 'Apr', '-04-01'
                                                             ,np.where(adjDate[0] == 'May', '-05-01'
                                                                      ,np.where(adjDate[0] == 'Jun', '-06-01'
                                                                               ,np.where(adjDate[0] == 'Jul', '-07-01'
                                                                                        ,np.where(adjDate[0] == 'Aug', '-08-01'
                                                                                                 ,np.where(adjDate[0] == 'Sep', '-09-01'
                                                                                                          ,np.where(adjDate[0] == 'Oct', '-10-01'
                                                                                                                   ,np.where(adjDate[0] == 'Nov', '-11-01'
                                                                                                                            ,np.where(adjDate[0] == 'Dec', '-12-01'
                                                                                                                                     , 0))))))))))))
adjDate['year'] = np.where(adjDate[1].astype(int) <= 18, '20'
                        ,np.where(adjDate[1].astype(int) > 18, '19'
                                 , 0))
adjDate['combined'] = adjDate['year'] + adjDate[1] + adjDate['month']
#adjDate.head()
SS['date'] = adjDate['combined']
#SS.head()
df = pd.merge(df,SS, how = 'inner', on = 'date')
#df.head()

### CPI - Consumer Price Index for All Urban Consumers: All Items
https://fred.stlouisfed.org/series/CPIAUCSL/

In [7]:
#CPI.head()
CPI.columns = ['date', 'cpi']
df = pd.merge(df,CPI, how = 'inner', on = 'date')
#df.head()

## Write to file

In [8]:
df.to_csv(path_or_buf = '~/Git/MachineLearningAndBigDataWithR/Data/assembled.csv', sep=',')